In [49]:
# Imports
import pathlib
import os
from dataclasses import dataclass, field
from typing import Dict, Literal, Optional, Tuple
import json

import imutils
import numpy as np
import cv2
from dataclasses_json import DataClassJsonMixin
import pandas as pd
from tqdm import tqdm

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent
DATA_DIR = GIT_ROOT / "data" / 'PhotosynthesisFall2022'

In [86]:
# Load the data
# vid_file = DATA_DIR / 'videos' / 'day 11' / "day11-screen-recording.mp4"
# vid_file = DATA_DIR / 'videos' / 'day 12' / "day12-screen-recording.mp4"
vid_file = DATA_DIR / 'videos' / 'day 13' / "day13-screen-recording.mp4"
assert vid_file.exists()
cap = cv2.VideoCapture(str(vid_file))
fps = cap.get(cv2.CAP_PROP_FPS)

# game_state_file = DATA_DIR / 'game_state' / 'day_11_game_state.csv'
# game_state_file = DATA_DIR / 'game_state' / 'day_12_game_state.csv'
game_state_file = DATA_DIR / 'game_state' / 'day_13_game_state.csv'
assert game_state_file.exists()
game_state_logs = pd.read_csv(game_state_file)

# Generate timestamp
game_state_logs['datetime'] = pd.to_datetime(game_state_logs['datetime'], format='%Y-%m-%d %H:%M:%S.%f')
game_state_logs['timestamp'] = (game_state_logs['datetime'] - game_state_logs['datetime'].iloc[0]).dt.total_seconds()

In [51]:
# Game state reconstruction routines
@dataclass
class Participant(DataClassJsonMixin):
    id: str
    position: Tuple[float, float]
    state: Literal['null', 'H2O', 'CO2', 'Sugar', 'O2', 'Thinking_H2O'] = 'null'

@dataclass
class EnvironmentState(DataClassJsonMixin):
    sun_state: Optional[bool] = None

@dataclass
class GameState(DataClassJsonMixin):
    participants: Dict[str, Participant] = field(default_factory=dict)
    environment: EnvironmentState = field(default_factory=EnvironmentState)

In [30]:
# Ceil for helping find information
for i, row in tqdm(game_state_logs.iterrows(). total=len(game_state_logs))):
    game_state = GameState.from_json(row.state)

    thinking_h2o_counts = 0
    for p in game_state.participants.values():
        if p.state == 'Thinking_H2O':
            thinking_h2o_counts += 1

    if thinking_h2o_counts >= 3:
        print(f"Found the row: {i}")
        break

Found the row: 10394


In [112]:
# Transformation constants
# CORRECTIONS = {'OFFSET': (-252,-280), 'AFFINE': (1.8,2)} # Day 11 and 12
CORRECTIONS = {'OFFSET': (-257,-280), 'AFFINE': (1.9,2.1)}

# TIME_OFFSET = 792.26 # Day 11
# TIME_OFFSET = 959.75 # Day 12
TIME_OFFSET = 4*60 + 30.9 # Day 13

START_INDEX = 1000
N = 20_000
RECORD = False

def xy_transforms(xy: Tuple[float, float], w: int, h:int) -> Tuple[float, float]:
    x, y = xy
    xx = int(((x+1)*h/2)*CORRECTIONS['AFFINE'][0] + CORRECTIONS['OFFSET'][0])
    yy = int(((y+1)*w/2)*CORRECTIONS['AFFINE'][1] + CORRECTIONS['OFFSET'][1])
    return (xx, yy)

def render(game_state: GameState, frame: np.ndarray):

    if game_state.participants:
        for p in game_state.participants.values():
            p.position = (eval(p.position[0]), eval(p.position[1]))
            xy = xy_transforms(p.position, frame.shape[0], frame.shape[1])
            frame = cv2.circle(frame, xy, 30, (0, 0, 255), 1)
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (0,0,0), 
                2, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (255,255,255), 
                1, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (0,0,0), 
                2,
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (255,255,255), 
                1,
                cv2.LINE_AA
            )

    return frame

# If you want to save the video
if RECORD:
    # video_file = DATA_DIR / 'time_alignment' / "day11-alignment.mp4"
    # video_file = DATA_DIR / 'time_alignment' / "day12-alignment.mp4"
    video_file = DATA_DIR / 'time_alignment' / "day13-alignment.mp4"
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
    out = cv2.VideoWriter(str(video_file),fourcc,fps,(650,566))

# Reset video and logs
cap.set(cv2.CAP_PROP_POS_FRAMES, START_INDEX)
game_state_pointer = 0

for i in tqdm(range(N), total=N):
    
    ret, frame = cap.read()
    if not ret:
        break

    frame_timestamp = (START_INDEX + i) / fps
    frame = cv2.putText(
                frame, 
                f"{pd.Timestamp(frame_timestamp, unit='s').strftime('%H:%M:%S.%f')}", 
                (5,30), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, 
                (0,0,255), 
                1, 
                cv2.LINE_AA
            )

    while game_state_logs['timestamp'].iloc[game_state_pointer+1] < frame_timestamp + TIME_OFFSET:
        game_state_pointer += 1
    game_state = GameState.from_json(game_state_logs.iloc[game_state_pointer].state)

    # For macro alignment
    frame = cv2.putText(
        frame, 
        f"{game_state_pointer}", 
        (5,60), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1, 
        (0,0,255), 
        1, 
        cv2.LINE_AA
    )

    frame = render(game_state, frame)
    cv2.imshow('frame', imutils.resize(frame, width=1000))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    if RECORD:
        out.write(frame)

cv2.destroyAllWindows()
if RECORD:
    out.release()

# Record meta data
# json_file = DATA_DIR/'time_alignment'/'day11_meta.json'
# json_file = DATA_DIR/'time_alignment'/'day12_meta.json'
json_file = DATA_DIR/'time_alignment'/'day13_meta.json'
json_data = {'time_offset': TIME_OFFSET, 'corrections': CORRECTIONS}
with open(json_file, 'w') as f:
    json.dump(json_data, f, indent=4)

# Save the data with the timestamp data
# aligned_logs_file = DATA_DIR/'time_alignment'/'day11_aligned_game_state.csv'
# aligned_logs_file = DATA_DIR/'time_alignment'/'day12_aligned_game_state.csv'
aligned_logs_file = DATA_DIR/'time_alignment'/'day13_aligned_game_state.csv'
game_state_logs['timestamp'] = game_state_logs['timestamp'] - TIME_OFFSET
aligned_state_logs = game_state_logs[game_state_logs['timestamp'] >= 0]
aligned_state_logs.timestamp = aligned_state_logs.timestamp.round(3)
aligned_state_logs.to_csv(aligned_logs_file, index=False)

  0%|          | 0/20000 [00:02<?, ?it/s]
/tmp/ipykernel_8421/3020369089.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aligned_state_logs.timestamp = aligned_state_logs.timestamp.round(3)
